In [ ]:
import os, cv2, dlib 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [ ]:
model = 'C:\\Users\\jimmy\\OneDrive\\Desktop\\Kuliah\\SKRIPSI\\Tugas Akhir\\Final\\Program_Jupyter\\shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(model)
detector = dlib.get_frontal_face_detector()
model_CNN = load_model('C:\\Users\\jimmy\\OneDrive\\Desktop\\Kuliah\\SKRIPSI\\Tugas Akhir\\Final\\Testing\\Rancangan_Model_19.h5')
model_CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 45, 45, 64)        1664      
                                                                 
 average_pooling2d (AverageP  (None, 9, 9, 64)         0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 64)          36928     
                                                                 
 average_pooling2d_1 (Averag  (None, 1, 1, 64)         0         
 ePooling2D)                                                     
                                                                 
 dropout (Dropout)           (None, 1, 1, 64)          0

In [ ]:
from mss import mss
import cv2
from PIL import Image
import numpy as np
from time import time



sct = mss()
loop_time = time()
while 1:
    mon = {'top': 0, 'left':0, 'width':560, 'height':420}
    begin_time = time()
    sct_img = sct.grab(mon)
    img = Image.frombytes('RGB', (sct_img.size.width, sct_img.size.height), sct_img.rgb)
    img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
    
    
    foto = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    
    landmark_tuple2 = []
    hasil2 = []
   
    rects = detector(foto.astype('uint8'), 0)
    if rects:
        for rect in rects:
            shape2 = predictor(foto, rect)
            for n in range(0, 27):
                x = shape2.part(n).x
                y = shape2.part(n).y
                landmark_tuple2.append((x, y))

            routes2 = []

            for i in range(16, -1, -1):
                routes2.append(landmark_tuple2[i])

            for i in range(17, 20):
                routes2.append(landmark_tuple2[i])

            for i in range(24, 27):
                routes2.append(landmark_tuple2[i])

            #img2 = cv2.cvtColor(foto, cv2.COLOR_BGR2RGB)

            mask2 = np.zeros((foto.shape[0], foto.shape[1]))
            mask2 = cv2.fillConvexPoly(mask2, np.array(routes2), 1)
            mask2 = mask2.astype(np.bool_)

            out2 = np.zeros_like(foto)
            out2[mask2] = foto[mask2]


            left = min(landmark_tuple2, key = lambda x:x[0])[0]
            right = max(landmark_tuple2, key = lambda x:x[0])[0]
            top = min(landmark_tuple2, key = lambda x:x[1])[1]
            bottom = max(landmark_tuple2, key = lambda x:x[1])[1]
            out2 = out2[max(0, top-10):bottom+10, max(0, left-10):right+10]

            out2 = cv2.resize(out2, (224, 224))

            hasil2.append(out2)
            
        hasil3 = hasil2[0]
        hasil3 = np.array(hasil3)
        hasil3 = hasil3 / 255.

        pre = model_CNN.predict(np.expand_dims(hasil3, axis=0))
        result = pre[0]
        result2 = ['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
        Result_Pre = result2[np.argmax(pre)]

        E1 = np.amax(result)
        M1 = result2[np.argmax(result)]
        print('Terdeteksi 1 = ', M1)
        indices = np.where(result==E1)
        result = np.delete(result, indices)
        result2.remove(M1)

        E2 = np.amax(result)
        M2 = result2[np.argmax(result)]
        print('Terdeteksi 2 = ', M2)
        indices2 = np.where(result==E2)
        result = np.delete(result, indices2)
        result2.remove(M2)

        E3 = np.amax(result)
        M3 = result2[np.argmax(result)]
        print('Terdeteksi 3 = ', M3)
        indices3 = np.where(result==E3)
        result = np.delete(result, indices3)
        result2.remove(M3)

        if (M1 == 'neutral' and M2 == 'surprise') or (M1 == 'surprise' and M2 == 'neutral'):  
            Final_Result = 'Confusion'
        elif (M1 == 'happiness' and M2 == 'neutral') or (M1 == 'neutral' and M2 == 'happiness'):
            Final_Result = 'Satisfaction'
        elif (M1 == 'sadness' and M2 == 'neutral') or (M1 == 'neutral' and M2 == 'sadness'):
            if M3 == 'anger':
                Final_Result = 'Frustrated'
            else:
                Final_Result = 'Disappointment'
        elif (M1 == 'sadness' and M2 == 'anger' and M3 == 'neutral') or (M1 == 'anger' and M2 == 'neutral' and M3 == 'sadness') or (M1 == 'anger' and M2 == 'sadness' and M3 == 'neutral') or (M1 == 'neutral' and M2 == 'anger' and M3 == 'sadness'):
            Final_Result = 'Frustrated'
        else:
            Final_Result = "UnKnown"
        
        print('Hasil = ', Final_Result)

        for rec2 in rects:
            x = rec2.left()
            y = rec2.top() 
            w = rec2.right()
            h = rec2.bottom()
        cv2.putText(img_bgr, Result_Pre, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1, cv2.LINE_AA)
#         cv2.putText(img_bgr, Final_Result, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1, cv2.LINE_AA)
        cv2.rectangle(img_bgr, (x, y), (w,h), (255,0,0), 2)

    
    fps = round(1 / (time()-loop_time), 2) 
    loop_time = time()
   # print('FPS {}'.format(1 / (time()-loop_time)))
    
    cv2.putText(img_bgr, str(fps), (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.imshow('Testing', np.array(img_bgr))
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
        


1/1 [==============================] - 0s 465ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 45ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 29ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 23ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 23ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 24ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [===================

1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  sadness
Terdeteksi 3 =  anger
Hasil =  Frustrated
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [=======

1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  happiness
Terdeteksi 3 =  neutral
Hasil =  UnKnown
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  happiness
Terdeteksi 3 =  neutral
Hasil =  UnKnown
1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  happiness
Hasil =  UnKnown
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  happiness
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  happiness
Hasil =  UnKnown
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  happiness
Terdeteksi 3 =  neutral
Hasil =  UnKnown
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1

1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  surprise
Terdeteksi 3 =  disgust
Hasil =  Confusion
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  surprise
Terdeteksi 3 =  disgust
Hasil =  Confusion
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [===

1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 16ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  surprise
Terdeteksi 3 =  neutral
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 16ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 18ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  surprise
Terdeteksi 3 =  neutral
Hasil =  UnKnown
1/1 [=

1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  surprise
Terdeteksi 3 =  disgust
Hasil =  Confusion
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 16ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  disgust
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 16ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [=================

1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 32ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 23ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 26ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  disgust
Terdeteksi 2 =  neutral
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [======================

1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 22ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  disgust
Hasil =  UnKnown
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  fear
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  fear
Terdeteksi 3 =  anger
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  fear
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  fear
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 20ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [=========================

1/1 [==============================] - 0s 23ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [==============================] - 0s 24ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  sadness
Hasil =  Frustrated
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 17ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  surprise
Terdeteksi 3 =  anger
Hasil =  Confusion
1/1 [==============================] - 0s 21ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  surprise
Terdeteksi 3 =  anger
Hasil =  Confusion
1/1 [==============================] - 0s 19ms/step
Terdeteksi 1 =  neutral
Terdeteksi 2 =  anger
Terdeteksi 3 =  surprise
Hasil =  UnKnown
1/1 [=======